In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl

In [2]:
default_colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]

In [3]:
registry = pd.read_csv("~/Box/dsi-core/11th-hour/idi-corporate-structure/20250630-0800-gleif-goldencopy-lei2-golden-copy.csv", low_memory=False)

In [4]:
sec2023 = pd.read_csv("../data/sec-2023.csv")

In [5]:
good_matches = pd.DataFrame([
    {"LEI": "HQD377W2YR662HK5JX27", "CIK": "0000001800"},
    {"LEI": "9C1X8XOOTYY2FNYTVH06", "CIK": "0000040533"},
    {"LEI": "3C7474T6CDKPR9K6YT90", "CIK": "0000040545"},
    {"LEI": "549300XMP3KDCKJXIU47", "CIK": "0000062709"},
    {"LEI": "549300ZFEEJ2IP5VME73", "CIK": "0000093751"},
    {"LEI": "WHENKOULSSK7WUM60H03", "CIK": "0000106640"},
    {"LEI": "PBJHNT51T5V6G5UQFV42", "CIK": "0000351569"},
    {"LEI": "529900E95812SYWMCE44", "CIK": "0000723612"},
    {"LEI": "ORQTRC074CWLT3DKHT41", "CIK": "0000740260"},
    {"LEI": "549300J4ZRR1SZF4ES33", "CIK": "0000783325"},
    {"LEI": "549300IGLYTZUK3PVP70", "CIK": "0000783325"},
    {"LEI": "FU4LY2G4933NH2E1CP29", "CIK": "0000796343"},
    {"LEI": "549300D3L3G0R4U4VT04", "CIK": "0000820313"},
    {"LEI": "549300LRGRD8XC9BTL66", "CIK": "0000851310"},
    {"LEI": "549300HWF93HB8BDM648", "CIK": "0000883237"},
    {"LEI": "784F5XWPLTWKTBV3E584", "CIK": "0000886982"},
    {"LEI": "549300VFZ8XJ9NUPU221", "CIK": "0001001250"},
    {"LEI": "SQL3F6CKNNBM3SQGHX24", "CIK": "0001020569"},
    {"LEI": "2T3D6M0JSY48PSZI1Q41", "CIK": "0001039684"},
    {"LEI": "P16VZZCKTJ5IONL6XE37", "CIK": "0001042729"},
    {"LEI": "5493000C01ZX7D35SD85", "CIK": "0001067983"},
    {"LEI": "549300UJLWOIWFDGB318", "CIK": "0001070081"},
    {"LEI": "549300VSMO9KYQWDND94", "CIK": "0001094285"},
    {"LEI": "54930073EK7NHVJF3D24", "CIK": "0001102432"},
    {"LEI": "549300DV9GIB88LZ5P30", "CIK": "0001103982"},
    {"LEI": "CUMYEZJOAF02RYZ1JJ85", "CIK": "0001126328"},
    {"LEI": "9695005J16R56PCL8N38", "CIK": "0001140536"},
    {"LEI": "549300WHC56FF48KL350", "CIK": "0001140536"},
    {"LEI": "529900EVVV534W8R0T32", "CIK": "0001234006"},
    {"LEI": "549300IH2S17MAIUCM32", "CIK": "0001273441"},
    {"LEI": "549300NPYMLM4NHTOF27", "CIK": "0001274494"},
    {"LEI": "MTLVN9N7JE8MIBIJ1H73", "CIK": "0001276187"},
    {"LEI": "549300D4549QKWETZ406", "CIK": "0001336917"},
    {"LEI": "549300NDDBVLJS24LD07", "CIK": "0001370755"},
    {"LEI": "549300YG7Z2BC73TCX59", "CIK": "0001403161"},
    {"LEI": "HL3H1H2BGXWVG3BSWR90", "CIK": "0001413329"},
    {"LEI": "724500M9BY5293JDF951", "CIK": "0001413447"},
    {"LEI": "5493007CT6ATBZ2L6826", "CIK": "0001433660"},
    {"LEI": "549300VDIGTMXUNT7H71", "CIK": "0001551182"},
    {"LEI": "5493005LXSLNF92BF097", "CIK": "0001572334"},
    {"LEI": "529900E7PUVK6BC8PE80", "CIK": "0001590364"},
    {"LEI": "549300TUSNXS3BFFML48", "CIK": "0001627014"},
    {"LEI": "549300CMHPBEY6VPOT75", "CIK": "0001674910"},
    {"LEI": "ISRPG12PN4EIEOEMW547", "CIK": "0001735707"},
    {"LEI": "549300GZKULIZ0WOW665", "CIK": "0001744489"},
    {"LEI": "2549006ALJE6NRO2Q503", "CIK": "0001764046"},
    {"LEI": "5493008G0Y8W4B2BWL04", "CIK": "0001825079"},
    {"LEI": "549300XR4L1D80AK4W76", "CIK": "0001834584"},
])